In [1]:
import json
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm, trange

import argparse

from env import Box, get_last_states
from sac_model import CirclePB
from sac_sampling import (
    sample_trajectories,
    evaluate_backward_logprobs,
)
from sac import SAC
from sac_replay_memory import ReplayMemory, trajectories_to_transitions

from utils import (
    fit_kde,
    plot_reward,
    sample_from_reward,
    plot_samples,
    estimate_jsd,
    plot_trajectories,
    plot_termination_probabilities,
)

import sac_config as config


parser = argparse.ArgumentParser()
parser.add_argument("--device", type=str, default=config.DEVICE)
parser.add_argument("--dim", type=int, default=config.DIM)
parser.add_argument("--delta", type=float, default=config.DELTA)
parser.add_argument("--R0", type=float, default=config.R0, help="Baseline reward value")
parser.add_argument("--R1", type=float, default=config.R1, help="Medium reward value (e.g., outer square)")
parser.add_argument("--R2", type=float, default=config.R2, help="High reward value (e.g., inner square)")
parser.add_argument("--reward_debug", action="store_true", default=config.REWARD_DEBUG)
parser.add_argument(
    "--reward_type",
    type=str,
    choices=["baseline", "ring", "angular_ring", "multi_ring", "curve", "gaussian_mixture"],
    default=config.REWARD_TYPE,
    help="Type of reward function to use. To modify reward-specific parameters (radius, sigma, etc.), edit rewards.py"
)
parser.add_argument(
    "--beta_min",
    type=float,
    default=config.BETA_MIN,
    help="Minimum value for the concentration parameters of the Beta distribution",
)
parser.add_argument(
    "--beta_max",
    type=float, 
    default=config.BETA_MAX,
    help="Maximum value for the concentration parameters of the Beta distribution",
)
parser.add_argument(
    "--PB",
    type=str,
    choices=["learnable", "tied", "uniform"],
    default=config.PB,
    help="Backward policy type",
)
parser.add_argument("--gamma_scheduler", type=float, default=config.GAMMA_SCHEDULER)
parser.add_argument("--scheduler_milestone", type=int, default=config.SCHEDULER_MILESTONE)
parser.add_argument("--seed", type=int, default=config.SEED)
parser.add_argument("--lr", type=float, default=config.LR, help="Learning rate for SAC")
parser.add_argument("--BS", type=int, default=config.BS)
parser.add_argument("--n_iterations", type=int, default=config.N_ITERATIONS)
parser.add_argument("--hidden_dim", type=int, default=config.HIDDEN_DIM)
parser.add_argument("--n_hidden", type=int, default=config.N_HIDDEN)
parser.add_argument("--n_evaluation_trajectories", type=int, default=config.N_EVALUATION_TRAJECTORIES)
parser.add_argument("--no_plot", action="store_true", default=config.NO_PLOT)
parser.add_argument("--no_wandb", action="store_true", default=config.NO_WANDB)
parser.add_argument("--wandb_project", type=str, default=config.WANDB_PROJECT)

# SAC-specific arguments
parser.add_argument("--tau", type=float, default=config.TAU, help="Tau for soft update")
parser.add_argument("--target_update_interval", type=int, default=config.TARGET_UPDATE_INTERVAL, help="Target network update interval")
parser.add_argument("--Critic_hidden_size", type=int, default=config.CRITIC_HIDDEN_SIZE, help="Hidden size for SAC critic networks")
parser.add_argument("--replay_size", type=int, default=config.REPLAY_SIZE, help="Replay buffer size")
parser.add_argument("--sac_batch_size", type=int, default=config.SAC_BATCH_SIZE, help="SAC batch size")
parser.add_argument("--updates_per_step", type=int, default=config.UPDATES_PER_STEP, help="SAC updates per step")

args = parser.parse_args([])


device = args.device
dim = args.dim
delta = args.delta
seed = args.seed
lr = args.lr
n_iterations = args.n_iterations
BS = args.BS

if seed == 0:
    seed = np.random.randint(int(1e6))

run_name = f"SAC_d{delta}_{args.reward_type}_PB{args.PB}_lr{lr}_sd{seed}"
run_name += f"_gamma{args.gamma_scheduler}_mile{args.scheduler_milestone}"
run_name += f"_batch{args.sac_batch_size}"
print(run_name)

SAC_d0.25_corner_squares_PBuniform_lr0.001_sd425779_gamma0.5_mile2500_batch256


In [ ]:
import os
import torch
import torch.nn.functional as F
from torch.optim import Adam
from sac_utils import soft_update, hard_update
from sac_model import QNetwork
from sac_model import CirclePF
from sac_sampling import sample_actions


class SAC(object):
    def __init__(self, args, env):

        self.gamma = 1.0  # Fixed to 1.0 for GFlowNet
        self.target_update_interval = args.target_update_interval
        self.device = env.device
        self.env = env  # Store env for sample_actions
        self.critic = QNetwork(env.dim, env.dim, args.Critic_hidden_size).to(device=self.device)
        self.critic_optim = Adam(self.critic.parameters(), lr=args.lr)
        self.critic_target = QNetwork(env.dim, env.dim, args.Critic_hidden_size).to(self.device)
        hard_update(self.critic_target, self.critic)
        self.policy = CirclePF(
            hidden_dim=args.hidden_dim,
            n_hidden=args.n_hidden,
            beta_min=args.beta_min,
            beta_max=args.beta_max,
        ).to(self.device)
        self.policy_optim = Adam(self.policy.parameters(), lr=args.lr)
        # Add scheduler for policy optimizer
        self.policy_scheduler = torch.optim.lr_scheduler.MultiStepLR(
            self.policy_optim,
            milestones=[i * args.scheduler_milestone for i in range(1, 10)],
            gamma=args.gamma_scheduler,
        )

    def update_parameters(self, memory, batch_size, updates):
        # Sample a batch from memory
        state_batch, action_batch, reward_batch, next_state_batch, done_batch = memory.sample(batch_size=batch_size)
        print(state_batch.shape)
        print(action_batch.shape)
        print(reward_batch.shape)
        print(next_state_batch.shape)
        print(done_batch.shape)
        # ★ 여기에서 그래프를 확실히 끊어주기
        state_batch      = state_batch.detach().to(self.device)
        action_batch     = action_batch.detach().to(self.device)
        reward_batch     = reward_batch.detach().to(self.device).unsqueeze(-1)   # (B, 1)로
        next_state_batch = next_state_batch.detach().to(self.device)
        done_batch       = done_batch.detach().to(self.device).unsqueeze(-1)     # (B, 1)로

        with torch.no_grad():
            # Problem 1: Handle sink states in next_state_batch
            # Identify non-sink next states
            non_sink_mask = ~torch.all(next_state_batch == self.env.sink_state, dim=-1)

            # Initialize with zeros (will be masked out anyway)
            next_state_action = torch.zeros_like(next_state_batch)
            next_state_log_pi = torch.zeros(next_state_batch.shape[0], device=self.device)

            # Only sample actions for non-sink next states
            if non_sink_mask.any():
                sampled_actions, sampled_log_pi = sample_actions(
                    self.env, self.policy, next_state_batch[non_sink_mask]
                )
                # Replace -inf terminal actions with zeros (won't be used anyway due to done_batch)
                sampled_actions = torch.where(  
                    torch.isinf(sampled_actions),
                    torch.zeros_like(sampled_actions),
                    sampled_actions
                )
                next_state_action[non_sink_mask] = sampled_actions
                next_state_log_pi[non_sink_mask] = sampled_log_pi

            # Replace sink states with zeros for critic input (won't be used due to done_batch)
            next_state_batch_clean = torch.where(
                torch.isinf(next_state_batch),
                torch.zeros_like(next_state_batch),
                next_state_batch
            )

            qf1_next_target, qf2_next_target = self.critic_target(next_state_batch_clean, next_state_action)
            min_qf_next_target = torch.min(qf1_next_target, qf2_next_target) - next_state_log_pi.unsqueeze(-1)
            # FIXED: done_batch is "done" (1=done, 0=not done)
            # Correct SAC Bellman: Q(s,a) = r + (1-done) * γ * Q(s',a')
            # Intermediate (done=0): Q = r + 1*γ*Q' = r + γ*Q' ✓
            # Terminal (done=1): Q = r + 0*γ*Q' = r ✓
            next_q_value = reward_batch + (1 - done_batch) * (min_qf_next_target)

        qf1, qf2 = self.critic(state_batch, action_batch)  # Two Q-functions to mitigate positive bias in the policy improvement step
        qf1_loss = F.mse_loss(qf1, next_q_value)  # JQ = 𝔼(st,at)~D[0.5(Q1(st,at) - r(st,at) - γ(𝔼st+1~p[V(st+1)]))^2]
        qf2_loss = F.mse_loss(qf2, next_q_value)  # JQ = 𝔼(st,at)~D[0.5(Q1(st,at) - r(st,at) - γ(𝔼st+1~p[V(st+1)]))^2]
        qf_loss = qf1_loss + qf2_loss

        self.critic_optim.zero_grad()
        qf_loss.backward()
        self.critic_optim.step()

        s0_mask = torch.all(state_batch == 0, dim=-1)
        non_s0_mask = ~s0_mask
        if s0_mask.any():
            pi_s0, log_pi_s0 = sample_actions(self.env, self.policy, state_batch[s0_mask])

        if non_s0_mask.any():
            pi_non_s0, log_pi_non_s0 = sample_actions(self.env, self.policy, state_batch[non_s0_mask])
            
        pi = torch.cat([pi_s0, pi_non_s0], dim=0)
        log_pi = torch.cat([log_pi_s0, log_pi_non_s0], dim=0)

        qf1_pi, qf2_pi = self.critic(state_batch, pi)
        min_qf_pi = torch.min(qf1_pi, qf2_pi)

        policy_loss = (log_pi - min_qf_pi).mean() # Jπ = 𝔼st∼D,εt∼N[α * logπ(f(εt;st)|st) − Q(st,f(εt;st))]

        self.policy_optim.zero_grad()
        policy_loss.backward()
        self.policy_optim.step()

        if updates % self.target_update_interval == 0:
            soft_update(self.critic_target, self.critic)

        return qf1_loss.item(), qf2_loss.item(), policy_loss.item()
        
    # Save model parameters
    def save_checkpoint(self, env_name, suffix="", ckpt_path=None):
        if not os.path.exists('checkpoints/'):
            os.makedirs('checkpoints/')
        if ckpt_path is None:
            ckpt_path = "checkpoints/sac_checkpoint_{}_{}".format(env_name, suffix)
        print('Saving models to {}'.format(ckpt_path))
        torch.save({'policy_state_dict': self.policy.state_dict(),
                    'critic_state_dict': self.critic.state_dict(),
                    'critic_target_state_dict': self.critic_target.state_dict(),
                    'critic_optimizer_state_dict': self.critic_optim.state_dict(),
                    'policy_optimizer_state_dict': self.policy_optim.state_dict()}, ckpt_path)

    # Load model parameters
    def load_checkpoint(self, ckpt_path, evaluate=False):
        print('Loading models from {}'.format(ckpt_path))
        if ckpt_path is not None:
            checkpoint = torch.load(ckpt_path)
            self.policy.load_state_dict(checkpoint['policy_state_dict'])
            self.critic.load_state_dict(checkpoint['critic_state_dict'])
            self.critic_target.load_state_dict(checkpoint['critic_target_state_dict'])
            self.critic_optim.load_state_dict(checkpoint['critic_optimizer_state_dict'])
            self.policy_optim.load_state_dict(checkpoint['policy_optimizer_state_dict'])

            if evaluate:
                self.policy.eval()
                self.critic.eval()
                self.critic_target.eval()
            else:
                self.policy.train()
                self.critic.train()
                self.critic_target.train()

In [ ]:
torch.manual_seed(seed)
np.random.seed(seed)

print(f"Using device: {device}")

env = Box(
    dim=dim,
    delta=delta,
    device_str=device,
    reward_type=args.reward_type,
    reward_debug=args.reward_debug,
    R0=args.R0,
    R1=args.R1,
    R2=args.R2,
)

# Get the true KDE
samples = sample_from_reward(env, n_samples=10000)
true_kde, fig1 = fit_kde(samples, plot=True)

# Create SAC agent (includes CirclePF as policy)
sac_agent = SAC(args, env)

# Create replay memory
memory = ReplayMemory(args.replay_size, seed, device=device)

bw_model = CirclePB(
    hidden_dim=args.hidden_dim,
    n_hidden=args.n_hidden,
    torso=sac_agent.policy.torso if args.PB == "tied" else None,
    uniform=args.PB == "uniform",
    beta_min=args.beta_min,
    beta_max=args.beta_max,
).to(device)

jsd = float("inf")
sac_updates = 0  # Track SAC update steps

for i in trange(1, n_iterations + 1):
    print(f"Iteration {i}")

    trajectories, actionss, logprobs, all_logprobs = sample_trajectories(
        env,
        sac_agent.policy,
        BS,
    )
    last_states = get_last_states(env, trajectories)
    logrewards = env.reward(last_states).log()
    bw_logprobs, all_bw_logprobs = evaluate_backward_logprobs(
        env, bw_model, trajectories
    )

    # Convert trajectories to transitions and push to replay memory
    all_states, all_actions, all_rewards, all_next_states, all_dones = trajectories_to_transitions(
        trajectories, actionss, all_bw_logprobs, logrewards, env
    )
    memory.push_batch(all_states, all_actions, all_rewards, all_next_states, all_dones)

    
    # TB (Trajectory Balance) loss
    if len(memory) > args.sac_batch_size:
        for _ in range(args.updates_per_step):
            qf1_loss, qf2_loss, policy_loss = sac_agent.update_parameters(memory, args.sac_batch_size, sac_updates)
            sac_updates += 1
        # Step the scheduler once per iteration (not per update)
        sac_agent.policy_scheduler.step()
    break
    if any(
        [
            torch.isnan(list(sac_agent.policy.parameters())[i]).any()
            for i in range(len(list(sac_agent.policy.parameters())))
        ]
    ):
        raise ValueError("NaN in model parameters")